In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import warnings
import torch
ct_utils = importlib.import_module('ct_utils');

warnings.filterwarnings("ignore");
print('Pandas version:', pd.__version__);
print('Numpy version:', np.__version__);
print('MatplotLib version:', mpl.__version__);
print('Sklearn version:', sklearn.__version__);
print('Seaborn version:', sns.__version__);

# Reading the datasets for train and test

### The following things need to be specified:

In [ ]:
melloddy_emb_cols = ["melloddy_emb_" + str(i) for i in range(2000)]; #columns specifying the structural encoding

time_cols = [ "time_" + str(i) for i in range(100)]; #timestamps of the measurments
conc_cols_po = [ "conc_po_" + str(i) for i in range(100)]; # po concentration measurments in nM 
conc_cols_iv = [ "conc_iv_" + str(i) for i in range(100)]; # iv concentration measurments in nM 

target_features =  time_cols + conc_cols_po + conc_cols_iv;

dose_column = "Dose" # mg/kg
mass_column = "Average Mass" # g/mol
species_colum = "Species" # Rat

input_data = "dummy.csv"
model_folder = "tmp_folder"

In [ ]:
# load the data
# each row is one experiment, i.e. one PK study
df = pd.read_csv(input_data)

In [ ]:
df["Dose_trf"] = ct_utils.recalc_dose(df[dose_column].to_numpy(dtype=np.float), df[mass_column].to_numpy(), df[species_colum].to_numpy())
df = df.dropna(subset = time_cols + conc_cols_po + conc_cols_iv, how="all")

In [ ]:
df_test = df.iloc[int(0.8*df.shape[0]):];
df_train = df.iloc[:int(0.8*df.shape[0])];
df_val = df_test.iloc[int(0.75*df_test.shape[0]):];
df_test = df_test.iloc[:int(0.75*df_test.shape[0])];

# Testing 

In [ ]:
import predict

#set the columns to be used as input features to the model (melloddy embeddings in this case). When additional features are used (e.g. the species) these have to be added here. Important: These have to be the same as used during training and also in the same order
additional_features = []; #no additional features in this case
predict.global_features = melloddy_emb_cols + additional_features

To predict the C-t curves, you need to run the follwing:

In [ ]:
tmp_times, po, iv = predict.predict_ct_from_ensemble(df_test.copy(), df_test["Dose_trf"].to_numpy(), model_folder, num_models=10, meas_times=None, num_cmpts=2);

Now, let's plot a few examples

In [ ]:
f, axs = plt.subplots(2, 4, figsize=(20,5));
axs = axs.ravel();
plt.subplots_adjust(wspace = 0.5);
plt.subplots_adjust(hspace = 0.5);

i = 0;
plot_i = 0;
while plot_i<8:
    
    #ca_consts_pred_po = preds[i, :7];
    #t_po_pred, po_curve_pred = ct_utils.ODE_solutions(ca_consts_pred_po, route="p.o.", dose=df_test["Dose_trf"].iloc[i]);
    
    t_po_pred = tmp_times[i, :];
    po_curve_pred = po[i, :];
    
    po_curve_pred= po_curve_pred[t_po_pred<=24]
    t_po_pred = t_po_pred[t_po_pred<=24]

    t_meas = df_test[time_cols].iloc[i].to_numpy();
    po_curve_true = df_test[conc_cols_po].iloc[i].to_numpy();

    
    if len(po_curve_true[~np.isnan(po_curve_true)]) == 0:
        i = i + 1;
        continue;
        
    sns.lineplot(x=t_po_pred, y=po_curve_pred, color="C1", linewidth=2, ax=axs[plot_i], label="pred");
    sns.scatterplot(x=t_meas, y=po_curve_true, color="C1", label="observed", ax=axs[plot_i]);
    axs[plot_i].set_yscale("log");
    axs[plot_i].set_xlabel("Time [hours]");
    axs[plot_i].set_ylabel("Concentration");

    i = i + 1;
    plot_i = plot_i + 1;

In [ ]:
f, axs = plt.subplots(2, 4, figsize=(20,5));
axs = axs.ravel();
plt.subplots_adjust(wspace = 0.5);
plt.subplots_adjust(hspace = 0.5);

i = 0;
plot_i = 0;
while plot_i<8:
    
    #ca_consts_pred_po = preds[i, :7];
    #t_po_pred, po_curve_pred = ct_utils.ODE_solutions(ca_consts_pred_po, route="p.o.", dose=df_test["Dose_trf"].iloc[i]);
    t_iv_pred = tmp_times[i, :];
    iv_curve_pred = iv[i, :];
    
    iv_curve_pred= iv_curve_pred[t_iv_pred<=24]
    t_iv_pred = t_iv_pred[t_iv_pred<=24]

    t_meas = df_test[time_cols].iloc[i].to_numpy();
    iv_curve_true = df_test[conc_cols_iv].iloc[i].to_numpy();

    
    if len(iv_curve_true[~np.isnan(iv_curve_true)]) == 0:
        i = i + 1;
        continue;
        
    sns.lineplot(x=t_iv_pred, y=iv_curve_pred, color="C0", linewidth=2, ax=axs[plot_i], label="pred");
    sns.scatterplot(x=t_meas, y=iv_curve_true, color="C0", label="observed", ax=axs[plot_i], s=20);
    axs[plot_i].set_yscale("log");
    axs[plot_i].set_xlabel("Time [hours]");
    axs[plot_i].set_ylabel("Concentration");
    
    i = i + 1;
    plot_i = plot_i + 1;

### Now compare to the readouts to from the predicted c-t curves to the derived ones

Let's get the predictions of derived PK parameters using NCA from the predicted C-t curves

In [ ]:
derived_params_pred = predict.predict_derived_from_ensemble(df_test, df_test["Dose_trf"].to_numpy(), model_folder, df_test["Species"].to_numpy(), num_models=10, num_cmpts=2);
cols = ['AUCinf_p.o._pred', 'Cmax_p.o._pred', "t_half_po", 'AUCinf_i.v._pred', "Cmax_iv_pred", 'T(12)_i.v._pred', 'In vivo CL_pred', 'Vss_pred', "F_pred", 'MRT_pred'];
df_test[cols] = derived_params_pred;

To test the performance of the predictions, you need to have the following readouts available in your dataset with the same column names:

In [ ]:
pk_parameter_cols = ['AUCinf_p.o.','Cmax_p.o.', 'AUCinf_i.v.','T(12)_i.v.', 'In vivo CL', 'Vss', "F", "MRT"];
df_test[pk_parameter_cols] = np.random.rand(df_test.shape[0], len(pk_parameter_cols))

In [ ]:
df_test["AUMC_iv_pred"] = df_test["MRT_pred"]*df_test['AUCinf_i.v._pred']
df_test["AUMC_iv"] = df_test["MRT"] + df_test["AUCinf_i.v."] #substraction because already logarithm
df_test["F_pred"] = (df_test['AUCinf_p.o._pred']/df_test['AUCinf_i.v._pred'])

In [ ]:
from scipy import stats

pairs = [["AUCinf_i.v._pred", "AUCinf_i.v."], ["AUMC_iv_pred", "AUMC_iv"], ['T(12)_i.v._pred', 'T(12)_i.v.'], ['In vivo CL_pred', 'In vivo CL'], ["Vss_pred", "Vss"], ['MRT_pred', "MRT"]];

f, axs = plt.subplots(1, len(pairs), figsize=(30, 5));

axs = axs.ravel();
plt.subplots_adjust(wspace = 0.5);
plt.subplots_adjust(hspace = 0.5);

for i, tmp_pair in enumerate(pairs):
    
    tmp_df = df_test.copy();
    tmp_df[tmp_pair[0]] = np.log(df_test[tmp_pair[0]]) 
    #tmp_df[tmp_pair[1]] = np.log(df_test[tmp_pair[1]])

    sns.kdeplot(data=tmp_df, x=tmp_pair[0], y=tmp_pair[1], ax=axs[i], color="black");
    #sns.regplot(data=tmp_df, x=tmp_pair[0], y=tmp_pair[1], ax=axs[i], color="black", scatter_kws={"s":2});
    sns.scatterplot(data=tmp_df, x=tmp_pair[0], y=tmp_pair[1], ax=axs[i], color="C1", s=10);

    from_curve = tmp_df[tmp_pair[0]].to_numpy();
    print(len(from_curve[~np.isnan(from_curve)]));

    drv = tmp_df[tmp_pair[1]].to_numpy();
    print(len(drv[~np.isnan(drv)]))
    
    from_curve_filt = from_curve[(~pd.isnull(from_curve))  &  (~pd.isnull(drv))  & (~np.isinf(from_curve))  &  (~np.isinf(drv))];
    drv_filt = drv[(~pd.isnull(from_curve)) & (~pd.isnull(drv)) & (~np.isinf(from_curve))  &  (~np.isinf(drv))];
    
    mini = np.min(from_curve_filt)
    maxi = np.max(from_curve_filt)    
    axs[i].plot(np.linspace(mini, maxi, 1000), np.linspace(mini, maxi, 1000), color="black");
    
    spearman_r = stats.spearmanr(from_curve_filt, drv_filt, nan_policy="omit");
    pearson_r = stats.pearsonr(from_curve_filt, drv_filt);
    mfc = np.exp(np.nanmedian(np.abs(from_curve_filt - drv_filt)));
    rmse = np.sqrt(np.nanmean((from_curve_filt - drv_filt)**2));
    r_2 = sklearn.metrics.r2_score(drv_filt, from_curve_filt);
    
    abs_log_errs = np.abs(from_curve_filt - drv_filt);
    frac_2fold = float(len(abs_log_errs[np.exp(abs_log_errs)<=2.0]))/len(abs_log_errs);
    frac_3fold = float(len(abs_log_errs[np.exp(abs_log_errs)<=3.0]))/len(abs_log_errs);

    axs[i].set_title("Spearman R: %.2f \n Pearson R: %.2f \n  R**2: %.2f \n MFC: %.2f \n RMSE: %.2f \n Frac. 2-fold: %.2f \n " % (spearman_r[0], pearson_r[0], r_2, mfc, rmse, frac_2fold));
    axs[i].set_title(tmp_pair[1], fontsize=18);
    axs[i].set_xlabel("predicted " , fontsize=14);
    axs[i].set_ylabel("measured " , fontsize=14);
    axs[i].tick_params(axis='both', which='major', labelsize=12);

In [ ]:
from scipy import stats
pairs = [["AUCinf_p.o._pred", "AUCinf_p.o."], ["Cmax_p.o._pred", "Cmax_p.o."]];

f, axs = plt.subplots(1, len(pairs), figsize=(17, 5));
axs = axs.ravel();
plt.subplots_adjust(wspace = 0.5);
plt.subplots_adjust(hspace = 0.5);

for i, tmp_pair in enumerate(pairs):
    
    tmp_df = df_test.copy();
    tmp_df[tmp_pair[0]] = np.log(df_test[tmp_pair[0]]) 
    #tmp_df[tmp_pair[1]] = np.log(df_test[tmp_pair[1]])

    sns.kdeplot(data=tmp_df, x=tmp_pair[0], y=tmp_pair[1], ax=axs[i], color="black");
    #sns.regplot(data=tmp_df, x=tmp_pair[0], y=tmp_pair[1], ax=axs[i], color="black", scatter_kws={"s":2});
    sns.scatterplot(data=tmp_df, x=tmp_pair[0], y=tmp_pair[1], ax=axs[i], color="C0", s=10);

    from_curve = tmp_df[tmp_pair[0]].to_numpy();
    print(len(from_curve[~np.isnan(from_curve)]));

    drv = tmp_df[tmp_pair[1]].to_numpy();
    print(len(drv[~np.isnan(drv)]))
    
    from_curve_filt = from_curve[(~pd.isnull(from_curve))  &  (~pd.isnull(drv))  & (~np.isinf(from_curve))  &  (~np.isinf(drv))];
    drv_filt = drv[(~pd.isnull(from_curve)) & (~pd.isnull(drv)) & (~np.isinf(from_curve))  &  (~np.isinf(drv))];
    
    mini = np.min(from_curve_filt)
    maxi = np.max(from_curve_filt)    
    axs[i].plot(np.linspace(mini, maxi, 1000), np.linspace(mini, maxi, 1000), color="black");
    
    spearman_r = stats.spearmanr(from_curve_filt, drv_filt, nan_policy="omit");
    pearson_r = stats.pearsonr(from_curve_filt, drv_filt);
    mfc = np.exp(np.nanmedian(np.abs(from_curve_filt - drv_filt)));
    rmse = np.sqrt(np.nanmean((from_curve_filt - drv_filt)**2));
    r_2 = sklearn.metrics.r2_score(drv_filt, from_curve_filt);
    
    abs_log_errs = np.abs(from_curve_filt - drv_filt);
    frac_2fold = float(len(abs_log_errs[np.exp(abs_log_errs)<=2.0]))/len(abs_log_errs);
    frac_3fold = float(len(abs_log_errs[np.exp(abs_log_errs)<=3.0]))/len(abs_log_errs);

    #axs[i].set_title("Spearman R: %.2f \n Pearson R: %.2f \n  R**2: %.2f \n MFC: %.2f \n RMSE: %.2f \n Frac. 2-fold: %.2f \n Frac. 3-fold: %.2f" % (spearman_r[0], pearson_r[0], r_2, mfc, rmse, frac_2fold, frac_3fold));
    axs[i].set_title(tmp_pair[1], fontsize=18);
    axs[i].set_xlabel("predicted " , fontsize=14);
    axs[i].set_ylabel("measured " , fontsize=14);
    axs[i].tick_params(axis='both', which='major', labelsize=12);

In [ ]:
tmp_df = df_test.copy();
tmp_df["F_pred"] = np.log(df_test["F_pred"])
tmp_df = tmp_df[(~pd.isnull(tmp_df["F"])) & (~pd.isnull(tmp_df["F_pred"]))  &  (~np.isinf(tmp_df["F"])) & (~np.isinf(tmp_df["F_pred"])) ]

sns.kdeplot(data=tmp_df, x="F_pred", y="F", color="black");
sns.scatterplot(data=tmp_df, x="F_pred", y="F", color="C2");

from_curve = tmp_df["F_pred"].to_numpy();
drv = tmp_df["F"].to_numpy();
from_curve_filt = from_curve[(~pd.isnull(from_curve))  &  (~pd.isnull(drv))  & (~np.isinf(from_curve))  &  (~np.isinf(drv))]
drv_filt = drv[(~pd.isnull(from_curve))  &  (~pd.isnull(drv))  & (~np.isinf(from_curve))  &  (~np.isinf(drv))]
    
mini = np.min(from_curve_filt);
maxi = np.max(from_curve_filt);

plt.plot(np.linspace(mini, maxi, 1000), np.linspace(mini, maxi, 1000), color="black");

spearman_r = stats.spearmanr(from_curve_filt, drv_filt, nan_policy="omit");
pearson_r = stats.pearsonr(from_curve_filt, drv_filt);
mfc = np.exp(np.nanmedian(np.abs(from_curve_filt - drv_filt)));
rmse = np.sqrt(np.nanmean((from_curve_filt - drv_filt)**2));
r_2 = sklearn.metrics.r2_score(drv_filt, from_curve_filt);
abs_log_errs = np.abs(from_curve_filt - drv_filt)
frac_2fold = float(len(abs_log_errs[np.exp(abs_log_errs)<=2.0]))/len(abs_log_errs);
frac_3fold = float(len(abs_log_errs[np.exp(abs_log_errs)<=3.0]))/len(abs_log_errs);

#plt.title("Spearman R: %.2f \n Pearson R: %.2f \n  R**2: %.2f \n MFC: %.2f \n RMSE: %.2f \n Frac. 2-fold: %.2f \n Frac. 3-fold: %.2f" % (spearman_r[0], pearson_r[0], r_2, mfc, rmse, frac_2fold, frac_3fold));
plt.title("F", fontsize=18)
plt.xlabel("predicted", fontsize=12);
plt.ylabel("measured", fontsize=12);
plt.tick_params(axis='both', which='major', labelsize=12);

### R**2 of curve fit

In [ ]:
#get data
meas_times = torch.tensor(df_test[time_cols].to_numpy());

tmp_times, po_curve_pred, iv_curve_pred = predict.predict_ct_from_ensemble(df_test.copy(), df_test["Dose_trf"].to_numpy(), 
                                                                           model_path, num_models=10, meas_times=meas_times, num_cmpts=2);

po_curve_meas = df_test[conc_cols_po].to_numpy();
iv_curve_meas = df_test[conc_cols_iv].to_numpy();

#calc r**2
r2_po = [];
r2_iv = [];
pearson_po = [];
pearson_iv = [];
spearman_po = [];
spearman_iv = [];
mfc_po = [];
mfc_iv = [];

for i in range(po_curve_meas.shape[0]):
    #t = tmp_times[i,:];
    tmp_po_meas = np.log( po_curve_meas[i,:]);
    tmp_iv_meas = np.log(iv_curve_meas[i,:]);
    
    tmp_po_pred = np.log(po_curve_pred[i,:].numpy());
    tmp_iv_pred = np.log(iv_curve_pred[i,:].numpy());
    tmp_times = meas_times[i,:].numpy();
    
    tmp_po_meas_filt = tmp_po_meas[(~pd.isnull(tmp_po_meas))  &  (~pd.isnull(tmp_po_pred))  & (~np.isinf(tmp_po_meas))  &  (~np.isinf(tmp_po_pred))];
    tmp_po_pred_filt = tmp_po_pred[(~pd.isnull(tmp_po_meas))  &  (~pd.isnull(tmp_po_pred))  & (~np.isinf(tmp_po_meas))  &  (~np.isinf(tmp_po_pred))];

    tmp_iv_meas_filt = tmp_iv_meas[(~pd.isnull(tmp_iv_meas))  &  (~pd.isnull(tmp_iv_pred))  & (~np.isinf(tmp_iv_meas))  &  (~np.isinf(tmp_iv_pred))];
    tmp_iv_pred_filt = tmp_iv_pred[(~pd.isnull(tmp_iv_meas))  &  (~pd.isnull(tmp_iv_pred))  & (~np.isinf(tmp_iv_meas))  &  (~np.isinf(tmp_iv_pred))];

    tmp_times_iv = tmp_times[(~pd.isnull(tmp_iv_meas))  &  (~pd.isnull(tmp_iv_pred))  & (~np.isinf(tmp_iv_meas))  &  (~np.isinf(tmp_iv_pred))];
    tmp_times_po = tmp_times[(~pd.isnull(tmp_po_meas))  &  (~pd.isnull(tmp_po_pred))  & (~np.isinf(tmp_po_meas))  &  (~np.isinf(tmp_po_pred))];
    
    if len(tmp_po_meas_filt)>5:
        r2_po.append(sklearn.metrics.r2_score(tmp_po_meas_filt, tmp_po_pred_filt));
        pearson_po.append(stats.pearsonr(tmp_po_meas_filt, tmp_po_pred_filt)[0]);
        spearman_po.append(stats.spearmanr(tmp_po_meas_filt, tmp_po_pred_filt)[0]);
        mfc_po.append(np.exp(np.nanmedian(np.abs(tmp_po_meas_filt - tmp_po_pred_filt))));

    if len(tmp_iv_meas_filt)>5:
        r2_iv.append(sklearn.metrics.r2_score(tmp_iv_meas_filt, tmp_iv_pred_filt));
        pearson_iv.append(stats.pearsonr(tmp_iv_meas_filt, tmp_iv_pred_filt)[0]);
        spearman_iv.append(stats.spearmanr(tmp_iv_meas_filt, tmp_iv_pred_filt)[0]);
        mfc_iv.append(np.exp(np.nanmedian(np.abs(tmp_iv_meas_filt - tmp_iv_pred_filt))));

df_plot = pd.DataFrame();
df_plot["R**2"] = list(r2_po) + list(r2_iv);
df_plot["Pearson R"] = list(pearson_po) + list(pearson_iv);
df_plot["Spearman R"] = list(spearman_po) + list(spearman_iv);
df_plot["MFC"] = list(mfc_po) + list(mfc_iv);
df_plot["Route"] = len(r2_po)*["p.o."] + len(r2_iv)*["i.v."];

In [ ]:
f, axs = plt.subplots(1, 4, figsize=(25, 5));
axs = axs.ravel();
plt.subplots_adjust(wspace = 0.3);
plt.subplots_adjust(hspace = 0.3);

#plotting
sns.histplot(data=df_plot, hue="Route", x="R**2", multiple="dodge", bins=20, ax=axs[0]);
sns.histplot(data=df_plot, hue="Route", x="Pearson R", multiple="dodge", bins=20, ax=axs[1]);
sns.histplot(data=df_plot, hue="Route", x="Spearman R", multiple="dodge", bins=20, ax=axs[2]);
sns.histplot(data=df_plot, hue="Route", x="MFC", multiple="dodge", bins=20, ax=axs[3], log_scale=True);

In [ ]:
f, axs = plt.subplots(1, 4, figsize=(25, 5));
axs = axs.ravel();
plt.subplots_adjust(wspace = 0.3);
plt.subplots_adjust(hspace = 0.3);

#plotting
sns.boxenplot(data=df_plot, x="Route", y="R**2",  ax=axs[0], showfliers=0);
sns.stripplot(data=df_plot, x="Route", y="R**2",  ax=axs[0], color="black", s=2);

sns.boxenplot(data=df_plot, x="Route", y="Pearson R",  ax=axs[1], showfliers=0);
sns.stripplot(data=df_plot, x="Route", y="Pearson R",  ax=axs[1], color="black", s=2);

sns.boxenplot(data=df_plot, x="Route", y="Spearman R", ax=axs[2], showfliers=0);
sns.stripplot(data=df_plot, x="Route", y="Spearman R",  ax=axs[2], color="black", s=2);

sns.boxenplot(data=df_plot, x="Route", y="MFC",ax=axs[3], showfliers=0);
sns.stripplot(data=df_plot, x="Route", y="MFC",  ax=axs[3], color="black", s=2);
axs[3].set_yscale("log");

## Predict multiple dosing schemes

In [ ]:
def get_multiDose_ct_profile(index, dosing_times, doses):


    times = np.arange(0,dosing_times[-1]+2, 0.1);
    final_po_curve = times*0.0;
    final_iv_curve = times*0.0;

    for i, tmp_dose_time in enumerate(dosing_times):

        #setup the input dataframe
        tmp_df = df_test.iloc[ind:(ind+1)];
        
        tmp_df["Dose"] = [doses[i]];
        tmp_df["Dose_trf"] = ct_utils.recalc_dose(tmp_df["Dose"].to_numpy(dtype=np.float), tmp_df["Average Mass"].to_numpy(), tmp_df["Species"].to_numpy());

        #predict the Ct profile for the current dose
        tmp_times, po, iv = predict.predict_ct_from_ensemble(tmp_df.copy(), tmp_df["Dose_trf"].to_numpy(), model_folder, 
                                                         num_models=10, meas_times=torch.tensor([times]), num_cmpts=2);

        for j in range(len(times)):
            if (j + int(tmp_dose_time/0.1)) >= len(times):
                break;

            final_po_curve[j + int(tmp_dose_time/0.1)] = final_po_curve[j + int(tmp_dose_time/0.1)] + po[0][j];
            final_iv_curve[j + int(tmp_dose_time/0.1)] = final_iv_curve[j + int(tmp_dose_time/0.1)] + iv[0][j];
            
    return times, final_po_curve, final_iv_curve;

In [ ]:
ind = 1256;
dosing_times = np.arange(0,121, 12)
doses = [1]*len(dosing_times);

times, final_po_curve, final_iv_curve = get_multiDose_ct_profile(ind, dosing_times, doses);

f, axs = plt.subplots(1, 2, figsize=(10, 4));
axs = axs.ravel();
plt.subplots_adjust(wspace = 0.2);
plt.subplots_adjust(hspace = 0.2);

axs[0].plot(times, final_po_curve, linewidth=5);
axs[1].plot(times, final_iv_curve, linewidth=5);

axs[0].set_xlabel("Time [h]");
axs[0].set_ylabel("Concentration [h]");

axs[1].set_xlabel("Time [h]");
axs[1].set_ylabel("Concentration [h]")

In [ ]:
ind = 1256;
dosing_times = np.arange(0,121, 6)
doses = [1]*len(dosing_times);

times, final_po_curve, final_iv_curve = get_multiDose_ct_profile(ind, dosing_times, doses);

f, axs = plt.subplots(1, 2, figsize=(10, 4));
axs = axs.ravel();
plt.subplots_adjust(wspace = 0.2);
plt.subplots_adjust(hspace = 0.2);

axs[0].plot(times, final_po_curve, linewidth=5);
axs[1].plot(times, final_iv_curve, linewidth=5);

axs[0].set_xlabel("Time [h]");
axs[0].set_ylabel("Concentration");

axs[1].set_xlabel("Time [h]");
axs[1].set_ylabel("Concentration");

In [ ]:
ind = 1256;
dosing_times = np.arange(0,121, 24)
doses = [1]*len(dosing_times);

times, final_po_curve, final_iv_curve = get_multiDose_ct_profile(ind, dosing_times, doses);

f, axs = plt.subplots(1, 2, figsize=(10, 4));
axs = axs.ravel();
plt.subplots_adjust(wspace = 0.2);
plt.subplots_adjust(hspace = 0.2);

axs[0].plot(times, final_po_curve, linewidth=5);
axs[1].plot(times, final_iv_curve, linewidth=5);

axs[0].set_xlabel("Time [h]");
axs[0].set_ylabel("Concentration");

axs[1].set_xlabel("Time [h]");
axs[1].set_ylabel("Concentration");